In [1]:
import torch
import tqdm
from transformers import AutoTokenizer, LlamaForCausalLM
#device =  'cuda:1'
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("./hf/7B")
model = LlamaForCausalLM.from_pretrained("./hf/7B",output_hidden_states=True)#.to(device)
# Encode the word 'slow' to obtain its token ID
input_ids = tokenizer.encode('slow', return_tensors='pt')#.to(device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [57]:
text_prototype = ['This time series exhibits an overall declining trend .',
                  'This time series shows an overall upward trend .',
                  'There is no apparent periodicity in this time series .',
                 'This time series displays clear periodicity .',
                 'The time series remains relatively stable with minimal fluctuations .',
                 'The time series undergoes significant instability over a period .',
                 'This time series is subject to strong noise interference .',
                 'This time series is not influenced by any noise interference .']

In [59]:
text_embedding = []
for sentence in text_prototype:
    print(f'sentence: {sentence} - last_hidden_state:')
    input_ids = tokenizer.encode(sentence, return_tensors='pt')
    with torch.no_grad():
        outputs = model(input_ids=input_ids)
    hidden_states = outputs['hidden_states'][-1][0,-1] #
    text_embedding.append(hidden_states)
    print(hidden_states)
    print(hidden_states.shape)

sentence: This time series exhibits an overall declining trend . - last_hidden_state:
tensor([ 3.6383, -1.0990,  0.7218,  ..., -0.5839,  0.4461, -0.2358])
torch.Size([4096])
sentence: This time series shows an overall upward trend . - last_hidden_state:
tensor([ 2.9130, -0.8803,  1.1642,  ..., -0.9045,  0.4519, -0.7690])
torch.Size([4096])
sentence: There is no apparent periodicity in this time series . - last_hidden_state:
tensor([ 5.0337, -0.9045,  0.5531,  ..., -0.9437,  0.4842, -0.8315])
torch.Size([4096])
sentence: This time series displays clear periodicity . - last_hidden_state:
tensor([ 4.3351, -0.9724,  0.7997,  ..., -0.7628,  0.9818, -0.5230])
torch.Size([4096])
sentence: The time series remains relatively stable with minimal fluctuations . - last_hidden_state:
tensor([ 3.1682, -0.5839,  1.3420,  ..., -0.4712, -0.4766, -0.8663])
torch.Size([4096])
sentence: The time series undergoes significant instability over a period . - last_hidden_state:
tensor([ 3.4644, -0.8719,  0.0507

In [70]:
# 
file = './'
text_embedding_tensor = torch.stack(text_embedding)

#
torch.save(text_embedding_tensor, file+'text_embedding.pt')


loaded_text_embedding_tensor = torch.load(file+'text_embedding.pt')
print(loaded_text_embedding_tensor.shape)

torch.Size([8, 4096])


In [63]:
# word_embedding_layer = model.model.embed_tokens

In [64]:
# words = ['slow','slower']
# for word in words:
#     input_ids = tokenizer.encode(word, return_tensors='pt')
#     embedding = word_embedding_layer(input_ids)
#     print(embedding)

In [65]:
# with torch.no_grad():
#     outputs = model(input_ids=input_ids)

In [66]:
# print(outputs.keys())

In [67]:
# print(len(outputs['hidden_states'])) # 

In [68]:
# for x in outputs['hidden_states']:
#     print(x.shape)

In [12]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
      (1): LlamaDecoderLayer(
        (self_attn): 

In [69]:
# prompt = "Say Hellow to me, Please."
# inputs = tokenizer(prompt, return_tensors="pt")

# # Generate
# generate_ids = model.generate( inputs.input_ids,pad_token_id=tokenizer.eos_token_id,max_length=30)
# tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]